##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [4]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Object Detection


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/s?q=google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1%20OR%20google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub models</a>
  </td>
</table>

This Colab demonstrates use of a TF-Hub module trained to perform object detection.

## Setup


In [5]:
from google.colab import drive
drive.mount('/content/drive')
gdrive_folder = 'drive/MyDrive'

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser 
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"
gdrive_folder = 'gdrive/My Drive'

In [ ]:
#@title Imports and function definitions

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time
import json

import os

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

## Example use

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [ ]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  # _, filename = tempfile.mkstemp(suffix=".jpg")
  # response = urlopen(url)
  # image_data = response.read()
  # image_data = BytesIO(image_data)
  pil_image = Image.open(url)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save("resized.jpeg", format="JPEG", quality=90)
  print("Image downloaded to %s." % "resized.jpeg")
  if display:
    display_image(pil_image)
  return "resized.jpeg"


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  #using left,right,top,bottom, calculate the area of the box
  area = abs(right - left) * abs(bottom - top)

  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin
  return area

def draw_boxes(image, boxes, class_names, scores, median_index, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  box_areas = []
  for i in range(min(boxes.shape[0], max_boxes)):
    if i == median_index:
    #if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      area = draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      box_areas.append(area)
      np.copyto(image, np.array(image_pil))
  return box_areas, image

## Apply module

Load a public image from Open Images v4, save locally, and display.

In [ ]:
# By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
#image_url = "/content/drive/MyDrive/AI/Disco_Diffusion/images_out/x10_ant_long3/x10_ant_long3(-1)_0969.png"  #@param
#downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

object_detection_request_directory = f'/content/{gdrive_folder}/AI/Object_Detection/request'


In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)
  image = img.numpy()
  image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
  im_width, im_height = image_pil.size
  result["box_areas"] = []
  for i in range(min(result["detection_boxes"].shape[0], 10)):
    if result["detection_scores"][i] >= 0.1:
      ymin, xmin, ymax, xmax = result["detection_boxes"][i]
      (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
      result["box_areas"].append(abs(left - right) * abs(top - bottom))
  import numpy
  median_index = numpy.argsort(result["box_areas"])[len(result["box_areas"])//2]

  ymin, xmin, ymax, xmax = result["detection_boxes"][i]
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
  real_left = min(left, right)
  real_right = max(left,right)
  real_top = min(bottom, top)
  real_bottom = max(bottom,top)
  box_width = real_right - real_left
  box_height = real_top - real_bottom
  hor_center = real_left + (box_width//2)
  ver_center = real_top + (box_height//2)
  print('im_height', im_height, im_width)
  print('winner', result["detection_scores"][median_index], result['box_areas'][median_index], real_left,real_right,real_top,real_bottom, hor_center, ver_center)

  coord_file = open(f"{object_detection_request_directory}/center_coordinates.txt", "w")
  coord_file.write(str(real_left) +','+ str(real_right) +','+str(real_top) +','+str(real_bottom))
  coord_file.close()

  box_areas, image_with_boxes = draw_boxes(
      image, result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"],median_index)

  display_image(image_with_boxes)

In [ ]:
import time
import sys 
timeout_minutes=60 #@param {type:"integer"}
#how to count the time that spends on while loop
start_time = time.time()

#if request_directory does not exist, create it
if not os.path.exists(object_detection_request_directory):
    os.makedirs(object_detection_request_directory)
while True:
  #while request_directory is empty
  while len(os.listdir(object_detection_request_directory)) == 0:
      time.sleep(1)
      if (time.time() - start_time) > timeout_minutes*60:
          print('timeout')
          sys.exit()

  #get the name of the file in the request_directory
  #print('os.listdir(request_directory)', os.listdir(request_directory))
  if len(os.listdir(object_detection_request_directory)) > 0:
    for file in os.listdir(object_detection_request_directory):
      if file.endswith('.png'):
        file_name = file
        resized = download_and_resize_image(f'{object_detection_request_directory}/{file_name}', 256, 256, False)
        run_detector(detector, resized)
        #delete the file in the request_directory
        os.remove(f'{object_detection_request_directory}/{file_name}')
        start_time = time.time()


NameError: name 'object_detection_request_directory' is not defined